In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import sys
import torchvision
torch.backends.cudnn.enabled = False

In [3]:
train_dataset = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ITMO_AI_HACK/Train-dataset.csv')
train_dataset.head()

,WELL,X,Y,MD,GR,RT,CN,DEN,DEPOSITIONAL_ENVIRONMENT,LITH_NAME,LITH_CODE
0,Well-1,0.8179,2.9814,1602.0,83.939,3.166,0.250,2.344,Marine,Marly sandstone,1200
1,Well-1,0.8179,2.9814,1602.1,84.166,3.135,0.246,2.352,Marine,Marly sandstone,1200
2,Well-1,0.8179,2.9814,1602.2,85.055,3.089,0.244,2.352,Marine,Marly sandstone,1200
3,Well-1,0.8179,2.9814,1602.3,86.352,3.042,0.242,2.355,Marine,Sandy marl,1300
4,Well-1,0.8179,2.9814,1602.4,87.614,3.003,0.241,2.369,Marine,Sandy marl,1300


In [4]:
train_dataset = train_dataset.drop(['WELL', 'LITH_NAME'], axis=1)
train_dataset.head()

,X,Y,MD,GR,RT,CN,DEN,DEPOSITIONAL_ENVIRONMENT,LITH_CODE
0,0.8179,2.9814,1602.0,83.939,3.166,0.250,2.344,Marine,1200
1,0.8179,2.9814,1602.1,84.166,3.135,0.246,2.352,Marine,1200
2,0.8179,2.9814,1602.2,85.055,3.089,0.244,2.352,Marine,1200
3,0.8179,2.9814,1602.3,86.352,3.042,0.242,2.355,Marine,1300
4,0.8179,2.9814,1602.4,87.614,3.003,0.241,2.369,Marine,1300


In [5]:
ohe = OneHotEncoder(sparse=False)
new_ohe_features = ohe.fit_transform(train_dataset.DEPOSITIONAL_ENVIRONMENT.values.reshape(-1,1))
tmp = pd.DataFrame(new_ohe_features, columns=['ENV' + 
                                             str(i) for i in range(new_ohe_features.shape[1])])
train_dataset = pd.concat([train_dataset, tmp], axis=1)
train_dataset = train_dataset.drop(['DEPOSITIONAL_ENVIRONMENT'], axis=1)
cols = train_dataset.columns.tolist()
cols = cols[0:7] + cols[8:11] + cols[7:8]
train_dataset = train_dataset[cols]
train_dataset

,X,Y,MD,GR,RT,CN,DEN,ENV0,ENV1,ENV2,LITH_CODE
0,0.8179,2.9814,1602.0,83.9390,3.1660,0.25000,2.3440,0.0,1.0,0.0,1200
1,0.8179,2.9814,1602.1,84.1660,3.1350,0.24600,2.3520,0.0,1.0,0.0,1200
2,0.8179,2.9814,1602.2,85.0550,3.0890,0.24400,2.3520,0.0,1.0,0.0,1200
3,0.8179,2.9814,1602.3,86.3520,3.0420,0.24200,2.3550,0.0,1.0,0.0,1300
4,0.8179,2.9814,1602.4,87.6140,3.0030,0.24100,2.3690,0.0,1.0,0.0,1300
...,...,...,...,...,...,...,...,...,...,...,...
45744,3.0000,0.0000,2275.2,103.6016,1.0789,0.29558,2.3783,0.0,1.0,0.0,400
45745,3.0000,0.0000,2275.3,102.8472,1.0683,0.29264,2.3651,0.0,1.0,0.0,400
45746,3.0000,0.0000,2275.4,102.5699,1.0790,0.29425,2.3531,0.0,1.0,0.0,400
45747,3.0000,0.0000,2275.5,102.7901,1.1045,0.30096,2.3430,0.0,1.0,0.0,400


In [6]:
X = train_dataset.iloc[:,0:-1]
y = train_dataset.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.67,
                                                    random_state=6,
                                                    stratify=y)

In [7]:
scaler = preprocessing.StandardScaler().fit(X_train)
train_dataset.iloc[:,0:-1] = scaler.transform(train_dataset.iloc[:,0:-1])
train_dataset

,X,Y,MD,GR,RT,CN,DEN,ENV0,ENV1,ENV2,LITH_CODE
0,-0.837657,1.409368,0.808620,-0.058873,-0.029915,-1.251057,0.933553,-0.794124,2.082126,-0.861181,1200
1,-0.837657,1.409368,0.808879,-0.044573,-0.030352,-1.303434,0.986150,-0.794124,2.082126,-0.861181,1200
2,-0.837657,1.409368,0.809139,0.011430,-0.031001,-1.329622,0.986150,-0.794124,2.082126,-0.861181,1200
3,-0.837657,1.409368,0.809398,0.093134,-0.031663,-1.355811,1.005873,-0.794124,2.082126,-0.861181,1300
4,-0.837657,1.409368,0.809657,0.172634,-0.032213,-1.368905,1.097918,-0.794124,2.082126,-0.861181,1300
...,...,...,...,...,...,...,...,...,...,...,...
45744,1.311023,-1.301154,2.553478,1.179770,-0.059345,-0.654222,1.159061,-0.794124,2.082126,-0.861181,400
45745,1.311023,-1.301154,2.553737,1.132247,-0.059495,-0.692719,1.072277,-0.794124,2.082126,-0.861181,400
45746,1.311023,-1.301154,2.553996,1.114778,-0.059344,-0.671637,0.993382,-0.794124,2.082126,-0.861181,400
45747,1.311023,-1.301154,2.554255,1.128650,-0.058984,-0.583775,0.926978,-0.794124,2.082126,-0.861181,400


In [8]:
X = train_dataset.iloc[:,0:-1]
y = train_dataset.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.67,
                                                    random_state=6,
                                                    stratify=y)

In [9]:
train = pd.concat([X_train, y_train], axis=1)
train

,X,Y,MD,GR,RT,CN,DEN,ENV0,ENV1,ENV2,LITH_CODE
45566,1.311023,-1.301154,2.507342,1.855667,-0.053474,0.107469,0.727768,-0.794124,2.082126,-0.861181,300
43776,1.311023,-1.301154,2.043395,0.649467,-0.015290,-1.126138,1.599559,-0.794124,2.082126,-0.861181,400
12212,-0.898215,0.501951,-0.461145,-0.175186,0.010588,-0.712491,-0.418840,1.259250,-0.480278,-0.861181,300
39472,1.311023,-1.301154,0.927847,-0.109842,-0.023590,-0.704373,0.605481,-0.794124,-0.480278,1.161196,1300
29800,1.311023,-1.301154,-1.579025,-0.661507,0.007375,0.602693,-0.829094,1.259250,-0.480278,-0.861181,600
...,...,...,...,...,...,...,...,...,...,...,...
7300,-1.038729,1.426278,-0.293709,0.392001,-0.029005,0.038597,0.660366,-0.794124,-0.480278,1.161196,300
34800,1.311023,-1.301154,-0.283082,0.839264,-0.026998,-0.167772,0.085431,1.259250,-0.480278,-0.861181,100
14313,-0.898215,0.501951,0.083411,0.238337,-0.025335,-0.431358,0.274122,-0.794124,-0.480278,1.161196,1300
9884,-0.898215,0.501951,-1.064536,0.981141,-0.029764,0.485761,-0.543099,1.259250,-0.480278,-0.861181,100


In [10]:
test = pd.concat([X_test, y_test], axis=1)
test

,X,Y,MD,GR,RT,CN,DEN,ENV0,ENV1,ENV2,LITH_CODE
29062,-0.352207,0.600048,0.875965,0.961902,-0.030282,-0.725193,1.456035,-0.794124,2.082126,-0.861181,400
4386,-1.038729,1.426278,-1.049762,-0.598329,-0.014223,0.440802,-0.381082,1.259250,-0.480278,-0.861181,500
20596,-0.352207,0.600048,-1.318325,0.885488,-0.030683,2.195082,-1.360649,1.259250,-0.480278,-0.861181,100
11424,-0.898215,0.501951,-0.665385,-0.839883,0.037464,-1.379380,-1.820542,1.259250,-0.480278,-0.861181,600
10682,-0.898215,0.501951,-0.857703,-0.602732,0.022184,-0.959841,-0.499050,1.259250,-0.480278,-0.861181,500
...,...,...,...,...,...,...,...,...,...,...,...
25615,-0.352207,0.600048,-0.017458,-2.291862,-0.014963,0.474107,-1.075707,-0.794124,-0.480278,1.161196,600
39221,1.311023,-1.301154,0.862791,0.458970,-0.038515,0.114409,0.585100,-0.794124,-0.480278,1.161196,100
43077,1.311023,-1.301154,1.862222,1.172186,-0.058252,0.578076,0.802061,-0.794124,2.082126,-0.861181,300
9505,-0.898215,0.501951,-1.162768,-0.893120,0.018092,-0.177723,-0.309701,1.259250,-0.480278,-0.861181,600


In [11]:
torch.cuda.is_available()

True

In [12]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

device = get_device()
print(device)   
print(torch.cuda.is_available())
print(torch.cuda.device_count())

cuda:0
True
1


In [13]:
class2idx = {
    100:0,
    200:1,
    300:2,
    400:3,
    500:4,
    600:5,
    800:6,
    1000:7,
    1100:8,
    1200:9,
    1300:10,
    1400:11,
    1500:12
}

idx2class = {v: k for k, v in class2idx.items()}
train['LITH_CODE'].replace(class2idx, inplace=True)
train


,X,Y,MD,GR,RT,CN,DEN,ENV0,ENV1,ENV2,LITH_CODE
45566,1.311023,-1.301154,2.507342,1.855667,-0.053474,0.107469,0.727768,-0.794124,2.082126,-0.861181,2
43776,1.311023,-1.301154,2.043395,0.649467,-0.015290,-1.126138,1.599559,-0.794124,2.082126,-0.861181,3
12212,-0.898215,0.501951,-0.461145,-0.175186,0.010588,-0.712491,-0.418840,1.259250,-0.480278,-0.861181,2
39472,1.311023,-1.301154,0.927847,-0.109842,-0.023590,-0.704373,0.605481,-0.794124,-0.480278,1.161196,10
29800,1.311023,-1.301154,-1.579025,-0.661507,0.007375,0.602693,-0.829094,1.259250,-0.480278,-0.861181,5
...,...,...,...,...,...,...,...,...,...,...,...
7300,-1.038729,1.426278,-0.293709,0.392001,-0.029005,0.038597,0.660366,-0.794124,-0.480278,1.161196,2
34800,1.311023,-1.301154,-0.283082,0.839264,-0.026998,-0.167772,0.085431,1.259250,-0.480278,-0.861181,0
14313,-0.898215,0.501951,0.083411,0.238337,-0.025335,-0.431358,0.274122,-0.794124,-0.480278,1.161196,10
9884,-0.898215,0.501951,-1.064536,0.981141,-0.029764,0.485761,-0.543099,1.259250,-0.480278,-0.861181,0


In [14]:
test['LITH_CODE'].replace(class2idx, inplace=True)
test

,X,Y,MD,GR,RT,CN,DEN,ENV0,ENV1,ENV2,LITH_CODE
29062,-0.352207,0.600048,0.875965,0.961902,-0.030282,-0.725193,1.456035,-0.794124,2.082126,-0.861181,3
4386,-1.038729,1.426278,-1.049762,-0.598329,-0.014223,0.440802,-0.381082,1.259250,-0.480278,-0.861181,4
20596,-0.352207,0.600048,-1.318325,0.885488,-0.030683,2.195082,-1.360649,1.259250,-0.480278,-0.861181,0
11424,-0.898215,0.501951,-0.665385,-0.839883,0.037464,-1.379380,-1.820542,1.259250,-0.480278,-0.861181,5
10682,-0.898215,0.501951,-0.857703,-0.602732,0.022184,-0.959841,-0.499050,1.259250,-0.480278,-0.861181,4
...,...,...,...,...,...,...,...,...,...,...,...
25615,-0.352207,0.600048,-0.017458,-2.291862,-0.014963,0.474107,-1.075707,-0.794124,-0.480278,1.161196,5
39221,1.311023,-1.301154,0.862791,0.458970,-0.038515,0.114409,0.585100,-0.794124,-0.480278,1.161196,0
43077,1.311023,-1.301154,1.862222,1.172186,-0.058252,0.578076,0.802061,-0.794124,2.082126,-0.861181,2
9505,-0.898215,0.501951,-1.162768,-0.893120,0.018092,-0.177723,-0.309701,1.259250,-0.480278,-0.861181,5


In [15]:
train_tensor = torch.tensor(train.values).to(device)
test_tensor = torch.tensor(test.values).to(device)

In [37]:
class MulticlassClassification(nn.Module):
    def __init__(self, num_feature, num_class):
        super(MulticlassClassification, self).__init__()
        
        self.layer_1 = nn.Linear(num_feature, 256)
        self.layer_2 = nn.Linear(256, 128)
        self.layer_3 = nn.Linear(128, 64)
        self.layer_out = nn.Linear(64, num_class) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        self.batchnorm1 = nn.BatchNorm1d(256)
        self.batchnorm2 = nn.BatchNorm1d(128)
        self.batchnorm3 = nn.BatchNorm1d(64)
        
    def forward(self, x):
        x = self.layer_1(x)
        x = self.batchnorm1(x)
        x = self.relu(x)
        
        x = self.layer_2(x)
        x = self.batchnorm2(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        
        x = self.layer_3(x)
        x = self.batchnorm3(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_out(x)
        
        return x

In [38]:
BATCH_SIZE = 16
LEARNING_RATE = 0.0032
NUM_FEATURES = len(train.columns)-1
NUM_CLASSES = 13
model = MulticlassClassification(num_feature = NUM_FEATURES, num_class=NUM_CLASSES)
model.double().to(device)
torch.save(model, './gdrive/MyDrive/Colab Notebooks/ITMO_AI_HACK/multiclass.pth')
loss_function = nn.CrossEntropyLoss()
#optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
optimizer = optim.ASGD(model.parameters(), lr=LEARNING_RATE)

In [39]:
writer = SummaryWriter('./gdrive/MyDrive/tensorboard/MulticlassClassification/')
running_test_loss = 0.0
running_train_loss = 0.0
#lr = 0.0032
#optimizer = optim.Adam(model.parameters(), lr=lr)
batch = 0
min_loss = 1000
was_loss = 100

trainset = torch.utils.data.DataLoader(train_tensor,
                                       batch_size=256,
                                       shuffle=True)
testset = torch.utils.data.DataLoader(test_tensor,
                                      batch_size=256,
                                      shuffle=True)

train_loader = DataLoader(dataset=train_tensor,
                          batch_size=BATCH_SIZE)
test_loader = DataLoader(dataset=test_tensor, batch_size=BATCH_SIZE)

for batch in tqdm(range(3000)):
    for data in trainset:
        X = data[:, 0:NUM_FEATURES]
        y = data[:, NUM_FEATURES:]
        optimizer.zero_grad()
        output = model(X.double())
        y = y.type(torch.LongTensor).to(device)
        loss = loss_function(output, y.squeeze())
        loss.backward()
        optimizer.step()
    train_loss = (loss.cpu().detach().numpy())
    for data in testset:
        X = data[:, 0:NUM_FEATURES]
        y = data[:, NUM_FEATURES:]
        output = model(X.double())
        y = y.type(torch.LongTensor).to(device)
        loss = loss_function(output, y.squeeze())
    test_loss = (loss.cpu().detach().numpy())
    running_test_loss += test_loss
    running_train_loss += train_loss
    if ((batch % 50) == 49):
        writer.add_scalars(f'loss',{
            'training':running_train_loss / 50,
            'test': running_test_loss / 50,
        }, batch+1)
        print('Train loss: ', running_train_loss / 50, '| Test_loss: ', running_test_loss / 50)
        if ((running_test_loss / 50) <  min_loss):#сохранение весов по среднему из 50 батчей
            min_loss = running_test_loss / 50
            torch.save(model.state_dict(), './gdrive/MyDrive/Colab Notebooks/ITMO_AI_HACK/MulticlassClassification.pth') # saving model
        running_train_loss = 0
        running_test_loss = 0

Train loss:  1.0779028068452925 | Test_loss:  1.0775245880552493
Train loss:  0.8711375710744766 | Test_loss:  0.8433116230309
Train loss:  0.8046941275779641 | Test_loss:  0.7815480753663442
Train loss:  0.7648447355112937 | Test_loss:  0.7466094268404398
Train loss:  0.7439025148542291 | Test_loss:  0.732888882475687
Train loss:  0.7126939113272099 | Test_loss:  0.7345928845127838
Train loss:  0.714459354922172 | Test_loss:  0.7059045882078309
Train loss:  0.6906090720131567 | Test_loss:  0.6878934597574781
Train loss:  0.6704212924808438 | Test_loss:  0.6740090587111754
Train loss:  0.6738194069764595 | Test_loss:  0.6630731770742834
Train loss:  0.6885150509722934 | Test_loss:  0.6659578588483575
Train loss:  0.6511872932409157 | Test_loss:  0.6694617382387663
Train loss:  0.6511035584047068 | Test_loss:  0.6498741134364842
Train loss:  0.6456188603143442 | Test_loss:  0.6547697074024227
Train loss:  0.6337421286106648 | Test_loss:  0.6499509056774531
Train loss:  0.635993308714992

In [52]:
##################################################################################

In [53]:
def corr_df(x, corr_val):
    '''
    Obj: Drops features that are strongly correlated to other features.
          This lowers model complexity, and aids in generalizing the model.
    Inputs:
          df: features df (x)
          corr_val: Columns are dropped relative to the corr_val input (e.g. 0.8)
    Output: df that only includes uncorrelated features
    '''

    # Creates Correlation Matrix and Instantiates
    corr_matrix = x.corr()
    iters = range(len(corr_matrix.columns) - 1)
    drop_cols = []

    # Iterates through Correlation Matrix Table to find correlated columns
    for i in iters:
        for j in range(i):
            item = corr_matrix.iloc[j:(j+1), (i+1):(i+2)]
            col = item.columns
            row = item.index
            val = item.values
            if val >= corr_val:
                # Prints the correlated feature set and the corr val
                print(col.values[0], "|", row.values[0], "|", round(val[0][0], 2))
                drop_cols.append(i)

    drops = sorted(set(drop_cols))[::-1]

    # Drops the correlated columns
    for i in drops:
        col = x.iloc[:, (i+1):(i+2)].columns.values
        df = x.drop(col, axis=1)

    return df

corr_train_dataset = corr_df(train_dataset, 0.6)
corr_train_dataset

DEN | MD | 0.64
ENV1 | MD | 0.74


,X,Y,MD,GR,RT,CN,ENV0,ENV1,ENV2,LITH_CODE
0,-0.837657,1.409368,0.808620,-0.058873,-0.029915,-1.251057,-0.794124,2.082126,-0.861181,1200
1,-0.837657,1.409368,0.808879,-0.044573,-0.030352,-1.303434,-0.794124,2.082126,-0.861181,1200
2,-0.837657,1.409368,0.809139,0.011430,-0.031001,-1.329622,-0.794124,2.082126,-0.861181,1200
3,-0.837657,1.409368,0.809398,0.093134,-0.031663,-1.355811,-0.794124,2.082126,-0.861181,1300
4,-0.837657,1.409368,0.809657,0.172634,-0.032213,-1.368905,-0.794124,2.082126,-0.861181,1300
...,...,...,...,...,...,...,...,...,...,...
45744,1.311023,-1.301154,2.553478,1.179770,-0.059345,-0.654222,-0.794124,2.082126,-0.861181,400
45745,1.311023,-1.301154,2.553737,1.132247,-0.059495,-0.692719,-0.794124,2.082126,-0.861181,400
45746,1.311023,-1.301154,2.553996,1.114778,-0.059344,-0.671637,-0.794124,2.082126,-0.861181,400
45747,1.311023,-1.301154,2.554255,1.128650,-0.058984,-0.583775,-0.794124,2.082126,-0.861181,400


In [76]:
X_corr = corr_train_dataset.iloc[:,0:-1]
y_corr = corr_train_dataset.iloc[:,-1]

X_corr_train, X_corr_test, y_corr_train, y_corr_test = train_test_split(X_corr, y_corr,
                                                                       train_size=0.8,
                                                                       random_state=6,
                                                                       stratify=y_corr)

In [60]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
rfc = RandomForestClassifier()
rfc.fit(X_corr_train, y_corr_train)
rfc.predict(X_corr_test)

array([100, 400, 400, ..., 500, 600, 400])

In [78]:
param_grid = { 
    'max_depth': ['None', 1],
    'n_estimators': [80, 100,150],
    'max_features': ['sqrt', 'auto'],
    'min_samples_split':[2,5],
    'min_samples_leaf':[1,2],
    'bootstrap': ['False', 'True'],
    'criterion' :['gini', 'entropy']
}
from sklearn.model_selection import GridSearchCV
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 3, n_jobs=-1)
CV_rfc.fit(X_corr_train, y_corr_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=15,
                                              max_features='sqrt',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=2,
                                              min_samples_split=23,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=600, n_jobs=None,
                                              o...
                                              random_state=None,

In [79]:
rfc_3 = CV_rfc.best_estimator_
CV_rfc.best_params_

{'bootstrap': 'True',
 'criterion': 'gini',
 'max_depth': 1,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 80}

In [80]:
f1_score(y_corr_test,rfc_3.predict(X_corr_test), average='macro' )

0.14931273042589926